---
title: SDCMap Tutorial with `sdcpy-map`
subtitle: Climatic Analysis Example (PDO vs NOAA NCEP/NCAR Air Temperature, monthly)
format:
  html:
    toc: true
    toc-location: right
    number-sections: false
    code-fold: true
    code-tools: true
    css: custom.css
execute:
  warning: false
  message: false
  echo: true
---

In this notebook we run a full, reproducible SDCMap workflow using `sdcpy-map` as an extension of `sdcpy`.

We will:

- use a configurable climate driver index,
- build a mapped-field monthly anomaly cube,
- derive a reference month from the selected driver,
- compute and interpret local and spatial SDC diagnostics,
- generate the compact 4-layer map product.


## Setup

We import analysis tools and define a baseline study design.

In this run we use the default package demo pair:

- driver dataset key: `pdo`,
- mapped field dataset key: `ncep_air`.

The same workflow supports other driver/field keys or user-provided inputs.


In [ ]:
#| label: setup
#| code-summary: "Import packages and define analysis constants"
from __future__ import annotations

import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Keep output compact during long loops.
os.environ.setdefault("TQDM_DISABLE", "1")

# Resolve project root robustly from current working directory.
ROOT = Path.cwd()
if not (ROOT / "src").exists() and (ROOT.parent / "src").exists():
    ROOT = ROOT.parent

sys.path.insert(0, str((ROOT / "src").resolve()))

from sdcpy import SDCAnalysis, compute_sdc

import sdcpy.core as _sdcpy_core

if not getattr(_sdcpy_core, "_sdcpy_map_tqdm_patched", False):
    def _silent_tqdm(*args, _orig_tqdm=_sdcpy_core.tqdm, **kwargs):
        kwargs["disable"] = True
        return _orig_tqdm(*args, **kwargs)

    _sdcpy_core.tqdm = _silent_tqdm
    _sdcpy_core._sdcpy_map_tqdm_patched = True

from sdcpy_map.config import SDCMapConfig
from sdcpy_map.datasets import (
    DEFAULT_DRIVER_DATASET_KEY,
    DEFAULT_FIELD_DATASET_KEY,
    DRIVER_DATASETS,
    FIELD_DATASETS,
    align_driver_to_field,
    fetch_public_example_data,
    grid_coordinates,
    load_coastline,
    load_driver_series,
    load_field_anomaly_subset,
)
from sdcpy_map.layers import compute_sdcmap_layers, save_layers_npz
from sdcpy_map.plotting import plot_layer_maps_compact

plt.rcParams["font.family"] = "DejaVu Sans Mono"
plt.rcParams["axes.titlesize"] = 11
plt.rcParams["axes.labelsize"] = 10
plt.rcParams["xtick.labelsize"] = 8
plt.rcParams["ytick.labelsize"] = 8

DATA_DIR = ROOT / ".data"
OUT_DIR = ROOT / ".output"
DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)


We define analysis parameters and dataset keys.


In [ ]:
# Driver and mapped-variable keys are exchangeable.
DRIVER_KEY = DEFAULT_DRIVER_DATASET_KEY
FIELD_KEY = DEFAULT_FIELD_DATASET_KEY

# One-year windows and half-year lag range for a monthly tutorial analysis.
FRAGMENT_SIZE = 12
N_PERMUTATIONS = 49
TWO_TAILED = False  # one-tailed workflow
MIN_LAG = -6
MAX_LAG = 6
ALPHA = 0.05
TOP_FRACTION = 0.25

# Multi-year period to capture stable large-scale variability.
TIME_START = "1990-01-01"
TIME_END = "2023-12-01"

# North Pacific / North America sector suitable for PDO + air-temperature diagnostics.
LAT_MIN = 20
LAT_MAX = 70
LON_MIN = -170
LON_MAX = -60
LAT_STRIDE = 1
LON_STRIDE = 1

# Diagnostic point for local SDC matrix inspection.
POINT_LAT, POINT_LON = 45.0, -140.0


## Download public data sources

`fetch_public_example_data` downloads the selected driver and mapped variable plus coastline geometry.
This keeps the workflow reproducible while allowing dataset-key swaps.


In [ ]:
#| label: download-data
#| code-summary: "Download selected driver/field datasets and coastline"
paths = fetch_public_example_data(
    data_dir=DATA_DIR,
    driver_key=DRIVER_KEY,
    field_key=FIELD_KEY,
)

print("Driver key:", DRIVER_KEY, "->", DRIVER_DATASETS[DRIVER_KEY].description)
print("Field key:", FIELD_KEY, "->", FIELD_DATASETS[FIELD_KEY].description)
for key, path in paths.items():
    print(f"{key:>10}: {path.name} ({path.stat().st_size / 1e6:.2f} MB)")


## Load and align driver + mapped field

This step creates the analysis-ready inputs:

- cleaned driver series,
- monthly anomalies of the mapped field,
- exact timestamp alignment,
- data-derived reference month (`peak_date`) from the driver maximum in-window.


In [ ]:
#| label: load-data
#| code-summary: "Load/align driver and mapped-field anomalies, then derive the reference month"
driver_full = load_driver_series(paths["driver"], config=SDCMapConfig(time_start="1900-01-01", time_end="2100-01-01"), driver_key=DRIVER_KEY)
driver = driver_full.loc[TIME_START:TIME_END]

peak_date = pd.Timestamp(driver.idxmax())
peak_value = float(driver.loc[peak_date])

mapped_field = load_field_anomaly_subset(
    paths["field"],
    config=SDCMapConfig(
        time_start=TIME_START,
        time_end=TIME_END,
        lat_min=LAT_MIN,
        lat_max=LAT_MAX,
        lon_min=LON_MIN,
        lon_max=LON_MAX,
        lat_stride=LAT_STRIDE,
        lon_stride=LON_STRIDE,
    ),
    field_key=FIELD_KEY,
)

driver = align_driver_to_field(driver, mapped_field)
coastline = load_coastline(paths["coastline"])

lats, lons = grid_coordinates(mapped_field)

print("Driver length:", len(driver))
print("Mapped field dims:", mapped_field.shape)
print(
    "Lat/Lon resolution:",
    abs(float(mapped_field.lat[1] - mapped_field.lat[0])),
    "x",
    abs(float(mapped_field.lon[1] - mapped_field.lon[0])),
    "degrees",
)
print("Time span:", str(mapped_field.time.values[0]), "->", str(mapped_field.time.values[-1]))
print(f"Derived reference month from driver max: {peak_date.date()} ({DRIVER_KEY}={peak_value:.2f})")


## Diagnostic plots before running the full map

We first inspect the inputs. This helps catch issues (wrong period, wrong domain, unexpected anomalies) before spending time on full SDCMap computation.


In [ ]:
#| label: fig-driver-series
#| fig-cap: "Driver time series in the analysis window, with the reference month (in-window maximum) marked."
fig, ax = plt.subplots(figsize=(11, 3.8))
ax.plot(driver.index, driver.values, lw=1.5)
ax.axvline(peak_date, ls="--", lw=1.2)
ax.scatter([peak_date], [peak_value], zorder=3)
ax.set_title(f"Driver series ({DRIVER_KEY})")
ax.set_xlabel("Date")
ax.set_ylabel("Index value")
plt.show()


This plot verifies that the data-derived reference month (maximum Niño3.4 anomaly in-window) is sensible for the episode we want to characterize.


In [ ]:
#| label: fig-domain-map
#| fig-cap: "Mapped-variable domain and selected local diagnostic point."
fig, ax = plt.subplots(figsize=(9.5, 4.2))
coastline.plot(ax=ax, color="none", edgecolor="black", linewidth=0.4)
ax.set_xlim(LON_MIN, LON_MAX)
ax.set_ylim(LAT_MIN, LAT_MAX)
ax.scatter([POINT_LON], [POINT_LAT], marker="x", s=70)
ax.set_title(f"Mapped-variable domain ({FIELD_KEY}) and local diagnostic point")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()


This ensures we are evaluating the intended region and verifies the point selected for local SDC diagnostics.

In [ ]:
#| label: fig-sst-snapshot
#| fig-cap: "Animation of mapped-variable anomalies from 6 months before to 6 months after the data-derived reference month."
#| code-summary: "Build and display a +/-6 month anomaly animation around the reference month"
#| code-fold: true
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation, PillowWriter

window_start = peak_date - pd.DateOffset(months=6)
window_end = peak_date + pd.DateOffset(months=6)
snap_window = mapped_field.sel(time=slice(window_start, window_end))
frame_times = pd.DatetimeIndex(snap_window.time.values)

if len(frame_times) < 2:
    raise ValueError("Not enough monthly steps to build animation in the +/-6 month window.")

fig, ax = plt.subplots(figsize=(9.5, 4.2))
first = snap_window.isel(time=0)
pcm = ax.pcolormesh(lons, lats, first.values, shading="auto", cmap="RdBu_r", vmin=-2, vmax=2)
coastline.plot(ax=ax, color="none", edgecolor="black", linewidth=0.4)
coastline.plot(ax=ax, color="black", edgecolor="black", linewidth=1, zorder=1)
ax.set_xlim(LON_MIN, LON_MAX)
ax.set_ylim(LAT_MIN, LAT_MAX)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
title = ax.set_title("")
plt.colorbar(pcm, ax=ax, label="Anomaly")


def _update(frame_idx: int):
    frame = snap_window.isel(time=frame_idx).values
    pcm.set_array(frame.ravel())
    frame_date = pd.Timestamp(frame_times[frame_idx]).date()
    month_offset = frame_idx - 6
    title.set_text(f"Mapped-field anomaly evolution ({frame_date}, offset {month_offset:+d} months)")
    return pcm, title


anim = FuncAnimation(
    fig,
    _update,
    frames=len(frame_times),
    interval=800,
    blit=False,
    repeat=True,
)

assets_dir = ROOT / "reports" / "assets"
assets_dir.mkdir(parents=True, exist_ok=True)
gif_path = assets_dir / "mapped_field_peak_window.gif"

anim.save(gif_path, writer=PillowWriter(fps=2))
plt.close(fig)

print(f"Saved animation: {gif_path}")
display(HTML('<img src="assets/mapped_field_peak_window.gif" alt="Mapped-variable anomaly animation">'))


## Finalize `SDCMapConfig` from exploratory diagnostics

After exploring inputs, we freeze the parameters used in all SDC computations.


In [ ]:
#| label: finalize-config
#| code-summary: "Build SDCMapConfig using the data-derived reference month"
config = SDCMapConfig(
    fragment_size=FRAGMENT_SIZE,
    n_permutations=N_PERMUTATIONS,
    two_tailed=TWO_TAILED,
    min_lag=MIN_LAG,
    max_lag=MAX_LAG,
    alpha=ALPHA,
    top_fraction=TOP_FRACTION,
    peak_date=peak_date.strftime("%Y-%m-%d"),
    time_start=TIME_START,
    time_end=TIME_END,
    lat_min=LAT_MIN,
    lat_max=LAT_MAX,
    lon_min=LON_MIN,
    lon_max=LON_MAX,
    lat_stride=LAT_STRIDE,
    lon_stride=LON_STRIDE,
)

print(f"Reference month from driver max: {config.peak_date} ({DRIVER_KEY}={peak_value:.2f})")
print(f"Permutation-tail mode: {'two-tailed' if config.two_tailed else 'one-tailed'}")


## Local SDC at one grid point

Before mapping all cells, we inspect one location to understand:
- the full window-to-window SDC matrix,
- the subset that survives significance filtering,
- how local dynamics compare to the driver series.


In [ ]:
#| label: local-sdc
#| code-summary: "Compute local SDC matrix at diagnostic point"
local_da = mapped_field.sel(lat=POINT_LAT, lon=POINT_LON, method="nearest")
local_ts = np.asarray(local_da.values, dtype=float)

sdc_df = compute_sdc(
    ts1=driver.to_numpy(dtype=float),
    ts2=local_ts,
    fragment_size=config.fragment_size,
    n_permutations=config.n_permutations,
    two_tailed=config.two_tailed,
    min_lag=config.min_lag,
    max_lag=config.max_lag,
)

sig = sdc_df.loc[(sdc_df["p_value"] <= config.alpha) & np.isfinite(sdc_df["r"])]
print("Total pairs:", len(sdc_df), "| Significant pairs:", len(sig))
sig.head()


In [ ]:
#| label: fig-local-timeseries
#| fig-cap: "Driver and local mapped-field anomaly series at the selected diagnostic point."
fig, ax = plt.subplots(figsize=(11, 3.8))
ax.plot(driver.index, driver.values, lw=1.3, label=f"Driver ({DRIVER_KEY})")
ax.plot(driver.index, local_ts, lw=1.2, label=f"Local {FIELD_KEY} anomaly")
ax.axvline(peak_date, ls="--", lw=1.0, label="Reference month")
ax.set_title("Local time series used for SDC")
ax.set_xlabel("Date")
ax.set_ylabel("Standardized units")
ax.legend(ncol=3, fontsize=8, loc="upper left")
plt.show()


The next matrix plot shows where high/low local SDC values appear across driver-window and local-window start indices, and how much gets discarded by significance masking.

In [ ]:
#| label: fig-local-sdc-matrices
#| fig-cap: "Local SDC matrices at the diagnostic point: all correlations (left) and significant-only values after masking with p <= alpha (right)."
r_mat = sdc_df.pivot(index="start_2", columns="start_1", values="r")
p_mat = sdc_df.pivot(index="start_2", columns="start_1", values="p_value")
sig_r = r_mat.mask(p_mat > config.alpha)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12.2, 4.8), constrained_layout=True)

im1 = ax1.imshow(r_mat.to_numpy(dtype=float), origin="lower", aspect="auto", cmap="RdBu_r", vmin=-1, vmax=1)
ax1.set_title("All local SDC correlations")
ax1.set_xlabel("Driver window start")
ax1.set_ylabel("Local window start")
plt.colorbar(im1, ax=ax1, fraction=0.046)

im2 = ax2.imshow(sig_r.to_numpy(dtype=float), origin="lower", aspect="auto", cmap="RdBu_r", vmin=-1, vmax=1)
ax2.set_title(f"Significant only (p <= {config.alpha})")
ax2.set_xlabel("Driver window start")
ax2.set_ylabel("Local window start")
plt.colorbar(im2, ax=ax2, fraction=0.046)

plt.show()


In [ ]:
#| label: fig-local-combi
#| fig-cap: "High-level `sdcpy.SDCAnalysis.combi_plot` view at the same point, combining SDC heatmap and aligned series diagnostics."
analysis = SDCAnalysis(
    ts1=pd.Series(driver.values, index=driver.index),
    ts2=pd.Series(local_ts, index=driver.index),
    fragment_size=config.fragment_size,
    n_permutations=config.n_permutations,
    two_tailed=config.two_tailed,
    min_lag=config.min_lag,
    max_lag=config.max_lag,
)

_ = analysis.combi_plot(
    alpha=config.alpha,
    xlabel=DRIVER_KEY,
    ylabel=f"Local {FIELD_KEY}",
    title="SDCAnalysis.combi_plot at selected point",
    figsize=(6.8, 6.8),
)
plt.show()


## Full SDCMap-style layers over the domain

Now we run the same SDC logic at every grid cell and aggregate results into map layers. In this notebook state, all p-values in this section use one-tailed permutation testing.

Interpretation reminder:
- `corr_mean`: dominant extreme-correlation strength,
- `lag_mean`: average lag timing,
- `driver_rel_time_mean`: when strong relations appear relative to reference date,
- `dominant_sign`: dominant polarity of the selected extremes.


In [ ]:
#| label: compute-layers
#| code-summary: "Compute one-tailed SDCMap-style layers and save NPZ"
layers = compute_sdcmap_layers(driver=driver, mapped_field=mapped_field, config=config)

for key in ["corr_mean", "lag_mean", "driver_rel_time_mean", "dominant_sign"]:
    n = int(np.isfinite(layers[key]).sum())
    print(f"{key:>24}: {n} finite cells")

npz_path = save_layers_npz(
    OUT_DIR / f"sdcmap_layers_{DRIVER_KEY}_{FIELD_KEY}.npz",
    layers=layers,
    lats=lats,
    lons=lons,
)
print("Saved:", npz_path)


In [ ]:
#| label: fig-spatial-layers
#| fig-cap: "Compact 4-panel SDCMap-style output for the selected driver and mapped variable. Panels summarize strength, lag, event-relative timing, and dominant sign."
#| code-summary: "Render compact 4-panel map with returned subplot handles"
#| code-fold: true
png_path = OUT_DIR / f"sdcmap_layers_{DRIVER_KEY}_{FIELD_KEY}.png"
fig, axes, cbar_axes = plot_layer_maps_compact(
    layers=layers,
    lats=lats,
    lons=lons,
    coastline=coastline,
    out_path=png_path,
    title=f"SDCMap-style layers ({DRIVER_KEY} vs {FIELD_KEY} anomalies)",
    return_handles=True,
)

# `axes` and `cbar_axes` stay available for custom tweaks after rendering.
print("Saved:", png_path)
plt.show()


The next plot helps quantify map coverage and value spread, so we can distinguish broad coherent signals from sparse/noisy outputs.

In [ ]:
#| label: fig-coverage-distributions
#| fig-cap: "Coverage and distribution diagnostics for SDCMap outputs: valid-cell mask (left) and histograms of key layer values (right)."
valid = np.isfinite(layers["corr_mean"]).astype(float)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4.3), constrained_layout=True)

pcm = ax1.pcolormesh(lons, lats, valid, shading="auto", cmap="Greens", vmin=0, vmax=1)
coastline.plot(ax=ax1, color="none", edgecolor="black", linewidth=0.4)
ax1.set_xlim(config.lon_min, config.lon_max)
ax1.set_ylim(config.lat_min, config.lat_max)
ax1.set_title("Valid-cell mask")
ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude")
plt.colorbar(pcm, ax=ax1, label="valid")

corr_vals = layers["corr_mean"][np.isfinite(layers["corr_mean"])]
lag_vals = layers["lag_mean"][np.isfinite(layers["lag_mean"])]
ax2.hist(corr_vals, bins=25, alpha=0.8, label="corr_mean", color="#4c72b0")
ax2.hist(lag_vals, bins=25, alpha=0.6, label="lag_mean", color="#dd8452")
ax2.set_title("Layer value distributions")
ax2.set_xlabel("Value")
ax2.set_ylabel("Count")
ax2.legend()

plt.show()


## Practical sensitivity checks

Before drawing climatic conclusions, re-run the workflow with small parameter changes and compare map stability:

- Shift the analysis period (for example, add/remove one year on each side).
- Compare `fragment_size=9`, `12`, and `15`.
- Tighten/relax lag bounds (for example, `-3..3` vs `-9..9`).
- Vary `top_fraction` and check whether dominant spatial patterns persist.
- Track how many valid cells remain after significance filtering (`alpha`).

If the same broad structures persist across these runs, interpretation is more robust.
